## Transfer Learning CNN Model

In [9]:
import numpy as np
import pandas as pd
from tensorflow import keras

from keras.models import Sequential
from keras.layers import InputLayer, Dense, Dropout, GlobalAveragePooling2D, Normalization
from keras.utils import image_dataset_from_directory
from keras.optimizers import Adam

from keras.applications import MobileNetV2, InceptionV3, DenseNet121, VGG19
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.losses import sparse_categorical_crossentropy

from keras.callbacks import EarlyStopping

In [13]:
# Constants used in the CNN model

data_dir = 'processed'

IMG_HEIGHT, IMG_WIDTH = 512, 512
TARGET_HEIGHT, TARGET_WIDTH = 224, 224

BATCH_SIZE = 32
VALIDATION_SPLIT = 0.2

NUM_CLASSES = 13
EPOCHS = 15

In [14]:
train_ds = image_dataset_from_directory(
    data_dir,
    validation_split=VALIDATION_SPLIT,
    subset="training",
    seed=42,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

val_ds = image_dataset_from_directory(
    data_dir,
    validation_split=VALIDATION_SPLIT,
    subset="validation",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
print(f"Art Styles (Class Names): {class_names}")

Found 42385 files belonging to 13 classes.
Using 33908 files for training.
Found 42385 files belonging to 13 classes.
Using 8477 files for validation.
Art Styles (Class Names): ['Academic_Art', 'Art_Nouveau', 'Baroque', 'Expressionism', 'Japanese_Art', 'Neoclassicism', 'Primitivism', 'Realism', 'Renaissance', 'Rococo', 'Romanticism', 'Symbolism', 'Western_Medieval']


In [5]:
base_model = MobileNetV2(weights='imagenet', 
                      include_top=False, 
                      input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
                      pooling=None)

base_model.trainable = True
total_layers = len(base_model.layers)

for layer in base_model.layers[:-30]:
    layer.trainable = False

for layer in base_model.layers:
    if isinstance(layer, keras.layers.BatchNormalization):
        layer.trainable = False

model = Sequential([
    InputLayer(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    Normalization(
        mean=[-1.]*3,
        variance=[1.]*3,
    ),
    base_model
    ])

model.add(GlobalAveragePooling2D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(NUM_CLASSES, activation='softmax'))

model.compile(
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],     
    optimizer=Adam(1e-3)
    )
    

/var/folders/rc/9nwzyr6x5zlgphpz33y5tcm80000gn/T/ipykernel_91408/2176082607.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet',


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [18]:
import tensorflow as tf
from keras import layers, models
from keras.applications import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.optimizers import Adam

IMG_HEIGHT = 224
IMG_WIDTH = 224
NUM_CLASSES = 13

train_ds = image_dataset_from_directory(
    data_dir,
    validation_split=VALIDATION_SPLIT,
    subset="training",
    seed=42,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

val_ds = image_dataset_from_directory(
    data_dir,
    validation_split=VALIDATION_SPLIT,
    subset="validation",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
print(f"Art Styles (Class Names): {class_names}")
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)

# =============== Base Model ===============
base_model = InceptionV3(
    weights="imagenet",
    include_top=False,
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
    pooling=None
)

# freeze 200 lyaers in beginning
for layer in base_model.layers[:200]:
    layer.trainable = False

# BN layer is frozen
for layer in base_model.layers:
    if isinstance(layer, layers.BatchNormalization):
        layer.trainable = False

# =============== Model ===============
inputs = layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
x = preprocess_input(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)

x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.3)(x)

outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = models.Model(inputs, outputs)

model.compile(
    optimizer=Adam(1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20
)



Found 42385 files belonging to 13 classes.
Using 33908 files for training.
Found 42385 files belonging to 13 classes.
Using 8477 files for validation.
Art Styles (Class Names): ['Academic_Art', 'Art_Nouveau', 'Baroque', 'Expressionism', 'Japanese_Art', 'Neoclassicism', 'Primitivism', 'Realism', 'Renaissance', 'Rococo', 'Romanticism', 'Symbolism', 'Western_Medieval']


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide_2 (TrueDivide)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract_2 (Subtract)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_v3 (Functional)       │ (None, 5, 5, 2048)     │    21,802,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 13)             │         1,677 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,361,901 (85.30 MB)

 Trainable params: 15,353,869 (58.57 MB)

 Non-trainable params: 7,008,032 (26.73 MB)

Epoch 1/20
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 2866s 3s/step - accuracy: 0.4759 - loss: 1.6085 - val_accuracy: 0.6179 - val_loss: 1.2055
Epoch 2/20
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 2975s 3s/step - accuracy: 0.6470 - loss: 1.1113 - val_accuracy: 0.7389 - val_loss: 0.8059
Epoch 3/20
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 2437s 2s/step - accuracy: 0.7330 - loss: 0.8398 - val_accuracy: 0.7438 - val_loss: 0.7619
Epoch 4/20
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 2004s 2s/step - accuracy: 0.8042 - loss: 0.6166 - val_accuracy: 0.8239 - val_loss: 0.5643
Epoch 5/20
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 1992s 2s/step - accuracy: 0.8599 - loss: 0.4373 - val_accuracy: 0.8555 - val_loss: 0.5061
Epoch 6/20
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 5765s 5s/step - accuracy: 0.9020 - loss: 0.3119 - val_accuracy: 0.8648 - val_loss: 0.5046
Epoch 7/20
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 1575s 1s/step - accuracy: 0.9227 - loss: 0.2469 - val_accuracy: 0.8822 - val_loss: 0.4528
Epoch 8/20
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 1576s 1s/step - accuracy: 0.9437 -

In [6]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization (Normalization)   │ (None, 512, 512, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 16, 16, 1280)   │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 13)             │           845 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,431,053 (9.27 MB)

 Trainable params: 1,683,789 (6.42 MB)

 Non-trainable params: 747,264 (2.85 MB)

In [7]:
earlystop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True,
    verbose=1
)

In [33]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[earlystop]
)


Epoch 1/15
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 2552s 2s/step - accuracy: 0.1507 - loss: 2.4326 - val_accuracy: 0.1574 - val_loss: 2.4099
Epoch 2/15
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 2776s 3s/step - accuracy: 0.1606 - loss: 2.4043 - val_accuracy: 0.1574 - val_loss: 2.4036
Epoch 3/15
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 2916s 3s/step - accuracy: 0.1602 - loss: 2.4040 - val_accuracy: 0.1574 - val_loss: 2.4038
Epoch 4/15
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 7081s 7s/step - accuracy: 0.1611 - loss: 2.4035 - val_accuracy: 0.1574 - val_loss: 2.4037
Epoch 5/15
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 3192s 3s/step - accuracy: 0.1604 - loss: 2.4028 - val_accuracy: 0.1574 - val_loss: 2.4038
Epoch 6/15
1060/1060 ━━━━━━━━━━━━━━━━━━━━ 3657s 3s/step - accuracy: 0.1610 - loss: 2.4034 - val_accuracy: 0.1574 - val_loss: 2.4036
Epoch 7/15
  34/1060 ━━━━━━━━━━━━━━━━━━━━ 33:55 2s/step - accuracy: 0.1566 - loss: 2.4155

KeyboardInterrupt: 

In [19]:
import os 

SAVED_MODEL_DIR = './art_style_inceptionV3.keras'

model.save(SAVED_MODEL_DIR)